In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

 
while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
 
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)
 
  model_dir = pathlib.Path(model_dir)/"saved_model"
 
  model = tf.saved_model.load(str(model_dir))
 
  return model
 
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

model_name = 'centernet_mobilenetv2fpn_512x512_coco17_od'
detection_model = load_model(model_name)

    

In [ ]:
print(detection_model.signatures['serving_default'])

In [ ]:
import playsound as p
import threading

#to generate the corresponding audio label of the detected object
def audioLabel(index):
    i=0
    path = "C:/Users/Raihan/computerVision/models/research/object_detection/audio_feedback/cocotext.txt"
    with open(path) as f:
        for line in f:
            line=f.read().splitlines()
            for items in line:
                content = items
                i+=1
                if i == index:
                    break
    return(content)

def run_inference_for_single_image(model, image):
  image = np.float32(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]
  print(input_tensor)
  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)
  print(output_dict)
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections
  

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
   
  return output_dict
delay = 0
def show_inference(model, frame):
  dim = (320,320)
  frame = cv2.resize(frame,dim)
  global delay
  delay+=1
  #take the frame from webcam feed and convert that to array
  image_np = np.array(frame)
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      #instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=5)
    
      
    
  if delay==20:
        delay=0
        filename = ''
        base = 'C:/Users/Raihan/computerVision/models/research/object_detection/audio_feedback/audio/'
        ext = '.mp3'
        for i in output_dict['detection_classes']:
            filename = audioLabel(i)
            file = base+filename+ext
            print(file)
            threading.Thread(target=p.playsound, args=(file,), daemon=True).start()

         
  return(image_np)

In [ ]:
import cv2
video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    re,frame = video_capture.read()
    Imagenp=show_inference(detection_model, frame)
    cv2.imshow('object detection', cv2.resize(Imagenp, (800,600)))
   
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()